In [16]:
print('มันจะไม่บ้ง')

มันจะไม่บ้ง


In [17]:
from pythainlp.tag import pos_tag

In [18]:
def extract_characters_pos(text: str) -> list[str]:
    """
    จงวิเคราะห์ข้อความทางกฎหมาย ทำหน้าที่อ่านข้อความและจัดประเภทความผิดของแต่ละบุคคลตามหมวดหมู่ที่กำหนดไว้ด้านล่าง และ ตามบุคคลที่กำหนดไว้ด้านล่างเท่านั้น:
Class 1: ไม่มีความผิดใดๆ เลย (No fault whatsoever)
Class 2: ลักทรัพย์หรือชิงทรัพย์ (Theft or robbery)
Class 3: กระทำโดยประมาท, ละเลย หรือไม่ได้ตั้งใจ ซึ่งทำให้เกิดความเสียหายต่อชีวิตหรือทรัพย์สิน (Negligence or unintentional harm)
Class 4: บุกรุกเคหสถานหรือบุกรุกเวลากลางคืน (Trespassing on a dwelling or at night)
Class 5: กระทำโดยเจตนา ก่อให้เกิดความเสียหายต่อร่างกาย, จิตใจ หรือทรัพย์สิน (Intentional harm)
Class 6: ฉ้อโกง, ปลอมแปลงเอกสาร หรือสิ่งของเพื่อการค้า (Fraud, forgery, counterfeiting)
Class 7: ทำให้ผู้อื่นเสียชีวิตไม่ว่าในรูปแบบใด (Causing death)

โปรดแสดงผลลัพธ์ในรูปแบบ:
[{'ชื่อบุคคล': [เลขคลาสของความผิด]}, ...]
ถ้าบุคคลใดไม่มีความผิด ให้ใส่ [1]
หากบุคคลหนึ่งกระทำผิดหลายข้อ ให้ใส่เลขหลายข้อใน list
ให้ใช้เฉพาะข้อมูลจากข้อความเท่านั้น ห้ามคาดเดาเกินสิ่งที่ระบุ
แยกชื่อบุคคลแต่ละคนให้ชัดเจน และใส่ผลลัพธ์เรียงตามลำดับที่ปรากฏในข้อความ
    """
    tokens = word_tokenize(text)
    tagged = pos_tag(tokens, engine="perceptron")
    # เก็บ token ที่ tag == NPR
    names = [tok for tok, tag in tagged if tag == "NPR"]
    # คืน unique ตามลำดับ
    seen = set(); uniq = []
    for n in names:
        if n not in seen:
            uniq.append(n); seen.add(n)
    return uniq

In [19]:
# ─── เซลล์ A: โหลดข้อมูลแล้ว parse answers ───
import pandas as pd
import ast

# ถ้าคุณใช้ train_half.csv กับ dev_half.csv ตามที่ทำไว้ก่อนหน้านี้
train_df = pd.read_csv(r"C:\miewmiew\Hackathon\Hack2\crime-charges-analysis\train_half.csv")
# หรือจะใช้ train.csv เต็ม ๆ ก็ได้ ถ้าต้องการข้อมูลทั้งหมด
# train_df = pd.read_csv("crime-charges-analysis/train.csv")

def parse_answers(cell: str) -> dict[str, list[int]]:
    raw = ast.literal_eval(cell)
    out = {}
    for d in raw:
        for p, cs in d.items():
            out[p] = ast.literal_eval(cs)
    return out

train_df["parsed"] = train_df["answers"].apply(parse_answers)

print("Loaded", len(train_df), "rows")  # ตรวจว่าโหลดสำเร็จ


Loaded 900 rows


In [20]:
# 1) แสดงตัวอย่าง story และ POS tags ใกล้ๆ
from pythainlp.tokenize import word_tokenize
from pythainlp.tag import pos_tag

# เลือก story ตัวแรกมาเช็ค
story = train_df["story"].iloc[0]
print("Story:\n", story, "\n")

tokens = word_tokenize(story)
tagged = pos_tag(tokens, engine="perceptron")
print("POS tags:\n", tagged)


Story:
 งิ่มเฮียงขโมยกระเป๋าจากตลาดแล้วยิงฆ่าลูกค้าในร้าน 

POS tags:
 [('งิ่ม', 'NCMN'), ('เฮีย', 'NCMN'), ('ง', 'NCMN'), ('ขโมย', 'NCMN'), ('กระเป๋า', 'NCMN'), ('จาก', 'RPRE'), ('ตลาด', 'NCMN'), ('แล้ว', 'XVAE'), ('ยิง', 'VACT'), ('ฆ่า', 'NCMN'), ('ลูกค้า', 'NCMN'), ('ใน', 'RPRE'), ('ร้าน', 'NCMN')]


In [21]:
import re
def extract_characters_regex(text):
    # จับคำไทย 2 ตัวขึ้นไปแล้วคัดกรองอีกที
    cand = re.findall(r"[ก-๙]{2,}", text)
    # ตัดคำสั้นๆ ไม่ใช่ชื่อ (คุณอาจปรับ list นี้เอง)
    stop = set(["และ","ที่","กับ","ใน","ไป","มา"])
    return [w for w in cand if w not in stop]


In [22]:
def extract_characters_pos(text: str) -> list[str]:
    tokens = word_tokenize(text)
    tagged = pos_tag(tokens, engine="perceptron")
    names = [tok for tok, tag in tagged if tag == "NPR"]
    if not names:
        # fallback ถ้าไม่มีชื่อจาก POS
        names = extract_characters_regex(text)
    # unique preserve
    seen = set(); uniq = []
    for n in names:
        if n not in seen:
            uniq.append(n); seen.add(n)
    return uniq


In [23]:
# ─── โหลดและ parse train_half.csv ───
import pandas as pd, ast
train_df = pd.read_csv(r"C:\miewmiew\Hackathon\Hack2\crime-charges-analysis\train_half.csv")
def parse_answers(cell):
    if not isinstance(cell, str): return {}
    raw = ast.literal_eval(cell)
    out = {}
    for d in raw:
        for p, cs in d.items():
            out[p] = ast.literal_eval(cs)
    return out

train_df["parsed"] = train_df["answers"].apply(parse_answers)

# ─── สร้าง Vocabulary ของชื่อทั้งหมด ───
all_names = set()
for parsed in train_df["parsed"]:
    all_names.update(parsed.keys())
# เปลี่ยนเป็น list เพื่อรักษาลำดับคงที่ (optional)
all_names = list(all_names)

# ─── ฟังก์ชันค้นชื่อจาก vocabulary ───
def extract_names_vocab(text: str) -> list[str]:
    """
    คืนเฉพาะรายชื่อตัวละครที่อยู่ใน all_names และปรากฏใน text
    ไม่คืนการกระทำหรือคำทั่วไป
    """
    found = [name for name in all_names if name in text]
    return found

# ─── ทดสอบดูผล ───
for story in train_df["story"].sample(5, random_state=0):
    print("Story:", story)
    print("Names:", extract_names_vocab(story))
    print("-"*40)


Story: กิมเหล่นฉ้อโกงประชาชนในงานตลาดด้วยการปลอมใบแจ้งหนี้และเรียกเก็บเงินจากผู้ซื้อ
Names: ['กา', 'การ', 'กิมเหล่น']
----------------------------------------
Story: เจตษรินทร์บุกรุกเข้าร้านอาหารและขโมยเงินสดจากตู้อัตโนมัติในเวลากลางคืน ขณะที่จังเว้งใช้มีดโจมตีผู้คนในงานเลี้ยงทำให้ขวัญนรีบาดเจ็บสาหัส
Names: ['จังเว้ง', 'กร', 'ขวัญนรี', 'โจม', 'เจตษรินทร์']
----------------------------------------
Story: แจ้งฮ้งกำลังนั่งทำการบ้านอยู่ที่บ้านขณะที่ง่วนเส็งเผลอหลับไป
Names: ['กา', 'การ', 'ง่วนเส็ง', 'แจ้งฮ้ง', 'กำลัง']
----------------------------------------
Story: กานนท์วางกองวัสดุไว้นอกจุดที่กำหนดด้วยความชุ่ยของเค้า ทำให้ตกลงใส่คนเดินผ่านจนเสียชีวิต
Names: ['กา', 'กานนท์']
----------------------------------------
Story: กาขับรถเร็วเกินกฎหมายกำหนดในหมู่บ้าน ทำให้เสียหลักชนรั้วบ้านของเกษวัต
Names: ['กา', 'นก', 'เกษวัต']
----------------------------------------


In [24]:
# ─── ก่อนนิยาม classify_text ───
# 1) รวบรวมชื่อตัวละครจาก train parsed answers
all_names = {p for parsed in train_df["parsed"] for p in parsed.keys()}
# 2) กรองชื่อเล็กเกินไป ทิ้งชื่อที่มีความยาว <= 2 ตัวอักษร (มักไม่ใช่ชื่อจริง)
all_names = {n for n in all_names if len(n) > 2}
# 3) เรียงชื่อจากยาว→สั้น เพื่อจับชื่อยาวก่อน ไม่แย่ง substring ชื่อสั้น
sorted_names = sorted(all_names, key=lambda s: len(s), reverse=True)

def extract_names_vocab(text: str) -> list[str]:
    """
    คืนรายชื่อจาก sorted_names ที่ substring ปรากฏใน text
    เรียงตามความยาวของชื่อ (ชื่อยาวก่อน) และ occurrence ใน sorted_names
    """
    found = []
    for name in sorted_names:
        if name in text:
            found.append(name)
    return found

# ─── ใน classify_text ให้เรียก extract_names_vocab แทน extract_characters_* ───

def classify_text(text: str) -> dict[str, list[int]]:
    # names = extract_characters_pos(text)       # คำสั่งเดิมที่ไม่ได้ผล
    names = extract_names_vocab(text)            # ใช้วิธีใหม่
    result = {n: [] for n in names}
    for sent in re.split(r"[\.!\?]", text):
        sl = sent.lower()
        detected = {cls for cls, kws in crime_keywords.items() if any(kw in sl for kw in kws)}
        if 7 in detected:
            detected.add(5) if 5 in detected else detected.add(3)
        for n in names:
            if n in sent:
                result[n].extend(detected)
    return {n: sorted(set(v)) or [1] for n, v in result.items()}


In [25]:
# ─── ตัวอย่างโค้ดทดสอบ extract_names_vocab ───

import pandas as pd
import ast
import re

# 1) โหลดข้อมูล train_half.csv และ parse answers
train_df = pd.read_csv(r"C:\miewmiew\Hackathon\Hack2\crime-charges-analysis\train_half.csv")

def parse_answers(cell):
    if not isinstance(cell, str):
        return {}
    raw = ast.literal_eval(cell)
    out = {}
    for d in raw:
        for name, charges in d.items():
            out[name] = ast.literal_eval(charges)
    return out

train_df["parsed"] = train_df["answers"].apply(parse_answers)

# 2) สร้าง vocabulary ของชื่อจาก parsed answers
all_names = {name 
             for parsed in train_df["parsed"] 
             for name in parsed.keys()
             if len(name) > 2}      # กรองชื่อสั้น <=2

# 3) เรียงชื่อจากยาวไปสั้น
sorted_names = sorted(all_names, key=lambda s: len(s), reverse=True)

# 4) ฟังก์ชันดึงชื่อจาก vocabulary-based matching
def extract_names_vocab(text: str) -> list[str]:
    found = [name for name in sorted_names if name in text]
    return found


# 5) ทดสอบกับ 5 เรื่องสุ่ม
for story in train_df["story"].sample(5, random_state=42):
    names = extract_names_vocab(story)
    print("Story:", story)
    print("Extracted names:", names)
    print("-" * 60)



Story: กรกนกพรรณวางแผนเปิดแก๊สรั่วในห้องพักแล้วจุดไฟเผา ขณะกิมเสงกำลังหลับ จนกิมเสงเสียชีวิต
Extracted names: ['กรกนกพรรณ', 'กิมเสง', 'กำลัง']
------------------------------------------------------------
Story: กลพจน์ถูกจับในคดีชิงทรัพย์เมื่อลอบแย่งกระเป๋าจากผู้หญิงในตลาดที่คนพลุกพล่าน
Extracted names: ['กลพจน์']
------------------------------------------------------------
Story: คะนินทรฉ้อโกงผ่านการลงทุนหุ้นปลอมในตลาดหลักทรัพย์โดยมีจรัสฤทธิ์ร่วมขบวนการด้วย ส่งผลให้กิจติมาสูญเงินไปมากมาย
Extracted names: ['จรัสฤทธิ์', 'กิจติมา', 'คะนินทร', 'การ']
------------------------------------------------------------
Story: กฤธสุพจน์ฉ้อโกงเงินจากจิตอร่ามผ่านระบบออนไลน์โดยปลอมเอกสาร
Extracted names: ['กฤธสุพจน์', 'จิตอร่าม']
------------------------------------------------------------
Story: คุ้มพันธ์ขับรถเมาแล้วชนคนเดินเท้าในงานตลาด ทำให้หลายคนบาดเจ็บ
Extracted names: ['คุ้มพันธ์']
------------------------------------------------------------


ดีเทคชื่อได้ดีที่สุดดดดดดดดดดดดดดดดดดดด

In [26]:
# ─── อัปเดตฟังก์ชัน extract_names_vocab ───

# 1) สร้างชุดคำที่ไม่ใช่ชื่อ (stop words)
stop_words = {"การ", "กำลัง"}  # เพิ่มคำทั่วไปที่ไม่ควรจับเป็นชื่อ

# 2) สมมติว่า sorted_names ถูกนิยามไว้ก่อนหน้าตามขั้นตอนก่อนหน้า
#    (ชื่อทั้งหมดจาก parsed answers เรียงจากยาว→สั้น)

def extract_names_vocab(text: str) -> list[str]:
    """
    - หา substring จาก sorted_names ที่ปรากฏใน text
    - กรองชื่อใน stop_words ทิ้ง
    - ถ้าชื่อสั้นเป็น substring ของชื่อยาวกว่า ให้ตัดชื่อสั้นทิ้ง
    """
    # ขั้นแรก: หา names ที่ match substring และไม่เป็น stop_words
    found = [n for n in sorted_names
             if n in text and n not in stop_words]

    # ขั้นที่สอง: กรองชื่อสั้นที่เป็นส่วนหนึ่งของชื่อยาวกว่า
    filtered = []
    for name in found:
        # ถ้าไม่มีชื่ออื่น (ยาวกว่า) ที่ครอบชื่อปัจจุบัน
        if not any(name != other and name in other for other in found):
            filtered.append(name)

    return filtered

# ─── ทดสอบดูผล ───
for story in train_df["story"].sample(5, random_state=0):
    names = extract_names_vocab(story)
    print("Story:   ", story)
    print("Extracted names:", names)
    print("-" * 60)


Story:    กิมเหล่นฉ้อโกงประชาชนในงานตลาดด้วยการปลอมใบแจ้งหนี้และเรียกเก็บเงินจากผู้ซื้อ
Extracted names: ['กิมเหล่น']
------------------------------------------------------------
Story:    เจตษรินทร์บุกรุกเข้าร้านอาหารและขโมยเงินสดจากตู้อัตโนมัติในเวลากลางคืน ขณะที่จังเว้งใช้มีดโจมตีผู้คนในงานเลี้ยงทำให้ขวัญนรีบาดเจ็บสาหัส
Extracted names: ['เจตษรินทร์', 'จังเว้ง', 'ขวัญนรี', 'โจม']
------------------------------------------------------------
Story:    แจ้งฮ้งกำลังนั่งทำการบ้านอยู่ที่บ้านขณะที่ง่วนเส็งเผลอหลับไป
Extracted names: ['ง่วนเส็ง', 'แจ้งฮ้ง']
------------------------------------------------------------
Story:    กานนท์วางกองวัสดุไว้นอกจุดที่กำหนดด้วยความชุ่ยของเค้า ทำให้ตกลงใส่คนเดินผ่านจนเสียชีวิต
Extracted names: ['กานนท์']
------------------------------------------------------------
Story:    กาขับรถเร็วเกินกฎหมายกำหนดในหมู่บ้าน ทำให้เสียหลักชนรั้วบ้านของเกษวัต
Extracted names: ['เกษวัต']
------------------------------------------------------------


In [27]:
for story in train_df["story"].sample(5, random_state=0):
    print(">>", extract_characters_pos(story))


>> ['กิมเหล่นฉ้อโกงประชาชนในงานตลาดด้วยการปลอมใบแจ้งหนี้และเรียกเก็บเงินจากผู้ซื้อ']
>> ['เจตษรินทร์บุกรุกเข้าร้านอาหารและขโมยเงินสดจากตู้อัตโนมัติในเวลากลางคืน', 'ขณะที่จังเว้งใช้มีดโจมตีผู้คนในงานเลี้ยงทำให้ขวัญนรีบาดเจ็บสาหัส']
>> ['แจ้งฮ้งกำลังนั่งทำการบ้านอยู่ที่บ้านขณะที่ง่วนเส็งเผลอหลับไป']
>> ['กานนท์วางกองวัสดุไว้นอกจุดที่กำหนดด้วยความชุ่ยของเค้า', 'ทำให้ตกลงใส่คนเดินผ่านจนเสียชีวิต']
>> ['กาขับรถเร็วเกินกฎหมายกำหนดในหมู่บ้าน', 'ทำให้เสียหลักชนรั้วบ้านของเกษวัต']


In [28]:
import pandas as pd
import ast
import re
from pythainlp.tokenize import word_tokenize

# โหลดและ parse answers
train_df = pd.read_csv(r"C:\miewmiew\Hackathon\Hack2\crime-charges-analysis\train_half.csv")
def parse_answers(cell):
    if not isinstance(cell, str): return {}
    raw = ast.literal_eval(cell)
    out = {}
    for d in raw:
        for p, cs in d.items():
            out[p] = ast.literal_eval(cs)
    return out
train_df["parsed"] = train_df["answers"].apply(parse_answers)

# สร้าง all_names และ sorted_names ตามที่ก่อนหน้า
all_names = {n for parsed in train_df["parsed"] for n in parsed.keys() if len(n)>2}
sorted_names = sorted(all_names, key=len, reverse=True)
stop_words = {"การ","กา","นก","กำลัง"}

def extract_names_vocab(text: str) -> list[str]:
    found = [n for n in sorted_names if n in text and n not in stop_words]
    # กรองชื่อสั้นที่เป็น substring ของชื่อยาวกว่า
    filtered = [n for n in found
                if not any(n!=m and n in m for m in found)]
    return filtered

# สร้าง DataFrame เก็บผล
results = []
for _, row in train_df.iterrows():
    names = extract_names_vocab(row["story"])
    results.append({
        "id": row["id"],
        "detected_names": names
    })
df_results = pd.DataFrame(results)

# ดู 10 แถวแรก
print(df_results.head(10))

# หรือจะบันทึกทั้งหมดออก CSV แล้วเปิดดู
df_results.to_csv("detected_names_all.csv", index=False, encoding="utf-8-sig")
print("Saved to detected_names_all.csv")


           id               detected_names
0  6AzyVcLTK0                  [งิ่มเฮียง]
1  KVG7hHQ1L8         [เจตน์ณัชนัย, คออีก]
2  7YGDGTUHqt  [กัมปณันท์, ขวัญชีวี, กำไล]
3  ENUjSSQbyL                [กัญจน์รัศม์]
4  4fz81U1iph                   [กัณฑรีย์]
5  dQsZVNbsFf    [กาญจน์นันท์, กัญจนรักษ์]
6  MMUMsX8Sb2              [กิจฒิภาศ, คนุ]
7  MIbOGKpTjs   [กิจปราณีต, ขวัญสรณ์, คาร]
8  Vd1eFRblyM        [กฤตย์จันทร์, กษมณัน]
9  UzpchpzFtS                    [กรวินท์]
Saved to detected_names_all.csv


In [29]:
import pandas as pd
import ast

# 1) โหลดและ parse ข้อมูล
train_df = pd.read_csv(r"C:\miewmiew\Hackathon\Hack2\crime-charges-analysis\train_half.csv")

def parse_answers(cell):
    if not isinstance(cell, str):
        return {}
    raw = ast.literal_eval(cell)
    out = {}
    for d in raw:
        for p, cs in d.items():
            out[p] = ast.literal_eval(cs)
    return out

train_df["parsed"] = train_df["answers"].apply(parse_answers)

# 2) สร้าง vocabulary ของชื่อตัวละคร (ชื่อยาว > 2)
all_names = {
    name for parsed in train_df["parsed"]
    for name in parsed.keys()
    if len(name) > 2
}
sorted_names = sorted(all_names, key=len, reverse=True)

# 3) กำหนด stop words และฟังก์ชันดึงชื่อ
stop_words = {"การ", "กา", "นก", "กำลัง"}

def extract_names_vocab(text: str) -> list[str]:
    # หา substring จาก sorted_names และกรอง stop words
    found = [n for n in sorted_names if n in text and n not in stop_words]
    # ตัดชื่อสั้นๆ ที่เป็น substring ของชื่อยาวกว่า
    filtered = [
        n for n in found
        if not any(n != m and n in m for m in found)
    ]
    return filtered

# 4) สร้าง DataFrame ผลลัพธ์
results = []
for _, row in train_df.iterrows():
    names = extract_names_vocab(row["story"])
    results.append({
        "id": row["id"],
        "story": row["story"],
        "detected_names": ", ".join(names)
    })
df_results = pd.DataFrame(results)

# 5) แสดงตัวอย่าง 5 แถวแรก
print(df_results.head(5))

# 6) บันทึกผลทั้งหมดออก CSV
df_results.to_csv("detected_names_with_story.csv", index=False, encoding="utf-8-sig")
print("Saved to detected_names_with_story.csv")


           id                                              story  \
0  6AzyVcLTK0  งิ่มเฮียงขโมยกระเป๋าจากตลาดแล้วยิงฆ่าลูกค้าในร้าน   
1  KVG7hHQ1L8  คออีกวางแผนใส่สารหนูในขนมปังที่เจตน์ณัชนัยชอบก...   
2  7YGDGTUHqt  กัมปณันท์บุกรุกเข้าไปในบ้านของกำไลแล้วขโมยเครื...   
3  ENUjSSQbyL  กัญจน์รัศม์ฉ้อโกงเงินจากบริษัทด้วยข้อมูลปลอมใน...   
4  4fz81U1iph                    กัณฑรีย์ยิงปืนข่มขู่ในงานเลี้ยง   

              detected_names  
0                  งิ่มเฮียง  
1         เจตน์ณัชนัย, คออีก  
2  กัมปณันท์, ขวัญชีวี, กำไล  
3                กัญจน์รัศม์  
4                   กัณฑรีย์  
Saved to detected_names_with_story.csv


In [30]:
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report
from sklearn.preprocessing import MultiLabelBinarizer

# ตัวอย่าง y_true และ y_pred เป็น multilabel format
y_true = [
    [1, 2],   # ตัวอย่างที่ 1 (เช่น Class 1, 2)
    [2, 3],   # ตัวอย่างที่ 2 (เช่น Class 2, 3)
    [1],      # ตัวอย่างที่ 3 (เช่น Class 1)
    [4, 5],   # ตัวอย่างที่ 4 (เช่น Class 4, 5)
]

y_pred = [
    [1, 2],   # การทำนายของโมเดล (เช่น Class 1, 2)
    [2, 3],   # การทำนายของโมเดล (เช่น Class 2, 3)
    [1],      # การทำนายของโมเดล (เช่น Class 1)
    [4, 6],   # การทำนายของโมเดล (เช่น Class 4, 6)
]

# แปลงให้เป็น binary indicator matrix โดยใช้ MultiLabelBinarizer
mlb = MultiLabelBinarizer()
y_true_bin = mlb.fit_transform(y_true)
y_pred_bin = mlb.transform(y_pred)

# คำนวณ Precision, Recall และ F1-Score
precision = precision_score(y_true_bin, y_pred_bin, average="macro")
recall = recall_score(y_true_bin, y_pred_bin, average="macro")
f1 = f1_score(y_true_bin, y_pred_bin, average="macro")

# แสดงผล
print(f"Precision: {precision:.3f}")
print(f"Recall: {recall:.3f}")
print(f"F1-Score: {f1:.3f}")

# แสดง classification report
# แปลง mlb.classes_ ให้เป็น list ของ string
class_names = [str(cls) for cls in mlb.classes_]

print("\nClassification Report:")
print(classification_report(y_true_bin, y_pred_bin, target_names=class_names))


Precision: 0.800
Recall: 0.800
F1-Score: 0.800

Classification Report:
              precision    recall  f1-score   support

           1       1.00      1.00      1.00         2
           2       1.00      1.00      1.00         2
           3       1.00      1.00      1.00         1
           4       1.00      1.00      1.00         1
           5       0.00      0.00      0.00         1

   micro avg       1.00      0.86      0.92         7
   macro avg       0.80      0.80      0.80         7
weighted avg       0.86      0.86      0.86         7
 samples avg       1.00      0.88      0.92         7



c:\miewmiew\Hackathon\Hack2\.venv\Lib\site-packages\sklearn\preprocessing\_label.py:909: UserWarning: unknown class(es) [6] will be ignored
  warnings.warn(
c:\miewmiew\Hackathon\Hack2\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\miewmiew\Hackathon\Hack2\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
